In [1]:
import pandas                as     pd
import numpy                 as     np
import scipy.stats           as     stats
import statsmodels.api       as     sm
import statsmodels.stats.api as     sms
from   statsmodels.compat    import lzip
import seaborn               as     sns
import matplotlib.pyplot     as     plt

In [2]:
feature_names          = ["cement", "slag", "ash", "water", "superplastic", "coarseagg","fineagg","age", "strength"]
cement_df              = pd.read_csv('D:/RRD/self_learning1/Concrete_Data.csv', names = feature_names, header = 0)
print(cement_df.shape)
print(cement_df.columns)

(1030, 9)
Index(['cement', 'slag', 'ash', 'water', 'superplastic', 'coarseagg',
       'fineagg', 'age', 'strength'],
      dtype='object')


In [3]:
X    = cement_df[["cement", "slag", "ash", "water", "superplastic", "coarseagg","fineagg","age"]]
y    = cement_df.strength

### Build the model using statsmodel using the entire data to check assumptions

In [4]:
X             = sm.add_constant(X) # Add an intercept to our model
model         = sm.OLS(y, X).fit() ## OLS(output, input)
predictions   = model.predict(X)

## Print the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               strength   R-squared:                       0.616
Model:                            OLS   Adj. R-squared:                  0.613
Method:                 Least Squares   F-statistic:                     204.3
Date:                Sun, 26 May 2019   Prob (F-statistic):          6.29e-206
Time:                        05:16:45   Log-Likelihood:                -3869.0
No. Observations:                1030   AIC:                             7756.
Df Residuals:                    1021   BIC:                             7800.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -23.3312     26.586     -0.878      0.380     -75.500      28.837
cement           0.1198      0.008     14.113      0.000       0.103       0.136
slag             0.1039      0.010     10.247      0.000       0.084       0.124
ash              0.0879      0.013      6.988      0.000       0.063       0.113
water           -0.1499      0.040     -3.731      0.000      -0.229      -0.071
superplastic     0.2922      0.093      3.128      0.002       0.109       0.476
coarseagg        0.0181      0.009      1.926      0.054      -0.000       0.037
fineagg          0.0202      0.011      1.887      0.059      -0.001       0.041
age              0.1142      0.005     21.046      0.000       0.104       0.125
==============================================================================
Omnibus:                        5.378   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.068   Jarque-Bera (JB):                5.304
Skew:                          -0.174   Prob(JB):                       0.0705
Kurtosis:                       3.045   Cond. No.                     1.06e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.06e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### 1) No outliers

Firstly we try to get the studentized residuals using get_influence( ). 

In [5]:
influence     = model.get_influence()  
resid_student = influence.resid_studentized_external

In [6]:
resid = pd.concat([X, pd.Series(resid_student,name = "Studentized Residuals")],axis = 1)
resid.head()

,const,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,Studentized Residuals
0,1.0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,2.575535
1,1.0,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,0.789183
2,1.0,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,-1.605709
3,1.0,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,-2.604117
4,1.0,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,-1.623736


If the absolute value of studentized residuals is more than 3 then that observation is considered as an outlier and hence should be removed.

In [7]:
X      = X [np.absolute(resid['Studentized Residuals'] < 3)] 
print(X[np.absolute(resid['Studentized Residuals'] > 3)]) 

Empty DataFrame
Columns: [const, cement, slag, ash, water, superplastic, coarseagg, fineagg, age]
Index: []


D:\Python\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


### There are no outliers

### 2) No multi-collinearity

https://www.listendata.com/2018/01/linear-regression-in-python.html

Multi-collinearity increases the estimate of standard error of regression coefficients which makes some variables statistically insignificant when they should be significant.

We can detect multi-collinearity by:
+ By plotting scatter plots between predictor variables to have a visual description of their relationship.
+ By calculating the correlation coefficients between the variables we learn the extent of multi-collinearity in the data.
+ By calculating the Variable Inflation Factor (VIF) for each variable. 
VIF measures how much the variance of an estimated regression coefficients increases if your predictors are correlated.  The higher the value of VIF for the regressor, the more it is highly correlated to other variables.

VIF for a predictor variable is given by $\frac{1}{1 - R^2}$.
Here we take one of the explanatory variables as the target variable and all others as independent variables. So we run a regression between one of those independent variables with remaining independent variables. 

####  Detecting and Removing Multicollinearity 

##### We use the statsmodels library to calculate VIF

In [8]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

[variance_inflation_factor(X.values, j) for j in range(1, X.shape[1])]

[7.470995663275609,
 7.279223994355755,
 6.171691046112063,
 6.986798451584242,
 2.966729775126993,
 5.0550733137224055,
 7.008195354068961,
 1.118448699183081]

#### The following variables have multi-collinearity as their value exceeds 5:

####  cement with 7.47
####  slag with 7.28
####  ash with 6.17
####  water with 6.99
####  coarseagg with 5.06
####  fineagg with 7.01

### Create a function to remove the collinear variables. 

We choose a threshold of 5 which means if VIF is more than 5 for a particular variable then that variable will be removed.

In [9]:
def calculate_vif(x):
    thresh = 5.0
    output = pd.DataFrame()
    k = x.shape[1]
    vif = [variance_inflation_factor(x.values, j) for j in range(x.shape[1])]
    for i in range(1,k):
        print("Iteration no.")
        print(i)
        print(vif)
        a = np.argmax(vif)
        print("Max VIF is for variable no.:")
        print(a)
        if vif[a] <= thresh :
            break
        if i == 1 :          
            output = x.drop(x.columns[a], axis = 1)
            vif = [variance_inflation_factor(output.values, j) for j in range(output.shape[1])]
        elif i > 1 :
            output = output.drop(output.columns[a],axis = 1)
            vif = [variance_inflation_factor(output.values, j) for j in range(output.shape[1])]
    return(output)


In [10]:
X_pure = calculate_vif(X) 

Iteration no.
1
[6720.486622087176, 7.470995663275609, 7.279223994355755, 6.171691046112063, 6.986798451584242, 2.966729775126993, 5.0550733137224055, 7.008195354068961, 1.118448699183081]
Max VIF is for variable no.:
0
Iteration no.
2
[15.411428112935946, 3.3342657581085846, 4.155598615063491, 82.49488340214151, 5.472489336059043, 85.31891255975485, 72.7457058119554, 1.6993804552810714]
Max VIF is for variable no.:
5
Iteration no.
3
[14.459403389463185, 3.3046967315337246, 3.96926752195452, 72.20293057684762, 5.398055318529211, 48.64620607932568, 1.6992685093159439]
Max VIF is for variable no.:
3
Iteration no.
4
[9.398248435680163, 2.080626417128508, 2.947311664391804, 2.937369758296058, 14.194212265847629, 1.577226952228047]
Max VIF is for variable no.:
4
Iteration no.
5
[2.815910084082646, 1.5271448218478205, 1.8831376286345198, 2.9193601686224637, 1.5503096086209325]
Max VIF is for variable no.:
3


In [11]:
X_pure.head()

,cement,slag,ash,superplastic,age
0,540.0,0.0,0.0,2.5,28
1,540.0,0.0,0.0,2.5,28
2,332.5,142.5,0.0,0.0,270
3,332.5,142.5,0.0,0.0,365
4,198.6,132.4,0.0,0.0,360


In [12]:
[variance_inflation_factor(X_pure.values, j) for j in range(1 ,X_pure.shape[1])]

[1.5271448218478205,
 1.8831376286345198,
 2.9193601686224637,
 1.5503096086209325]

[variance_inflation_factor(train_out.values, j) for j in range(1,train_out.shape[1])]

### 3) Constant variance

Checking heteroscedasticity Using Goldfeld Quandt we test for heteroscedasticity.
Null Hypothesis: Error terms are homoscedastic
Alternative Hypothesis: Error terms are heteroscedastic.

In [13]:
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
name = ['F statistic', 'p-value']
test = sms.het_goldfeldquandt(model.resid, model.model.exog)
lzip(name, test)

[('F statistic', 0.4874620776756259), ('p-value', 0.9999999999999993)]

The p-value is 0.999 hence we can say that the residuals have constant variance. 

### 4) No autocorrelation

#### Checking for autocorrelation To ensure the absence of autocorrelation we use Ljungbox test.

####  Null Hypothesis: Autocorrelation is absent.
#### Alternative Hypothesis: Autocorrelation is present.

In [14]:
from statsmodels.stats import diagnostic as diag
diag.acorr_ljungbox(model.resid, lags = 1) 

(array([130.91396082]), array([2.58574305e-30]))

Since p-value is 0.1602 thus we can accept the null hypothesis and can say that autocorrelation is absent.

### 5) Normality of the residuals

#### Checking normality of residuals We use Shapiro Wilk test  from scipy library to check the normality of residuals.
#### Null Hypothesis: The residuals are normally distributed.
#### Alternative Hypothesis: The residuals are not normally distributed.

In [15]:
from scipy import stats
stats.shapiro(model.resid)

(0.9953246712684631, 0.002998962765559554)

https://www.listendata.com/2018/01/linear-regression-in-python.html

### 6) Linearity